In [1]:
# Célula 1: Instalação das dependências
!pip install spectral
!pip install pysptools
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.0/249.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 44.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysptools: filename=pysptools-0.15.0-py3-none-any.whl size=8133730 sha256=f6ac4988509f8a8b36e2bee5ab7c016a1e477dea24bd5937c8f852d2ecc8bda3
  Stored in directory: /root/.cache/pip/wheels/6c/cb/cf/19cafb99543a8aab05d24d64f6d0664a781e061ede8f30b619
Successfully built pysptools


In [2]:
# Célula 2: Importações e Carregamento de Dados Reais
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, cohen_kappa_score
import spectral
import pandas as pd # Import pandas
import os # Import os for listing files

# --- Carregamento de Dados Reais ---
# Caminho para o diretório contendo os datasets exportados do Google Earth Engine
# A code to search for files recursively within subdirectories.
data_dir_path = '/content/drive/My Drive/GEE_Exports_Pixel_Grouped'

all_files = []
if os.path.isdir(data_dir_path):
    print(f"Procurando arquivos no diretório e subdiretórios: {data_dir_path}")
    for root, dirs, files in os.walk(data_dir_path):
        for file in files:
            # Assumindo que os arquivos de dados terminam com '.csv'
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                all_files.append(file_path)
    print(f"Encontrados {len(all_files)} arquivos .csv no diretório e subdiretórios.")
else:
    print(f"ERRO: O caminho '{data_dir_path}' não é um diretório válido.")
    all_files = [] # Ensure all_files is empty if path is not a directory


data_df_list = []
if all_files:
    print("Lendo e combinando arquivos...")
    for i, file_path in enumerate(all_files):
        try:
            print(f"  Lendo arquivo {i+1}/{len(all_files)}: {os.path.basename(file_path)}")
            # Assumindo que os arquivos são CSVs. Ajuste 'sep' e 'header' se necessário.
            # Use low_memory=False para evitar warnings com datasets grandes
            df = pd.read_csv(file_path, low_memory=False)
            data_df_list.append(df)
            print(f"    Arquivo {os.path.basename(file_path)} lido com sucesso.")
        except Exception as e:
            print(f"    ERRO ao ler arquivo {os.path.basename(file_path)}: {e}")

    if data_df_list:
        # Concatenar todos os DataFrames lidos em um único DataFrame
        data_df = pd.concat(data_df_list, ignore_index=True)
        print("\nTodos os arquivos lidos e combinados com sucesso.")
        print("Dimensões do DataFrame combinado:", data_df.shape)
        print("Primeiras 5 linhas do DataFrame combinado:")
        display(data_df.head())

        # --- Identificação dos Dados Espectrais ---
        # Precisamos identificar quais colunas no DataFrame correspondem às bandas espectrais.
        # Assumindo que as colunas espectrais são numéricas e não são colunas de ID, lat, lon, etc.
        # Este é um passo crucial e pode precisar de ajuste manual dependendo do formato do seu GEE export.

        # Exemplo genérico: remover colunas comuns de metadados GEE se existirem
        metadata_cols = ['system:index', '.geo'] # Adicione outras colunas de metadados aqui se necessário
        spectral_cols = [col for col in data_df.columns if col not in metadata_cols and np.issubdtype(data_df[col].dtype, np.number)]

        if not spectral_cols:
            raise ValueError("Não foi possível identificar colunas espectrais no DataFrame combinado. Por favor, verifique o formato dos arquivos.")

        hsi_data = data_df[spectral_cols].values.astype(np.float32) # Convert to float32 for spectral library compatibility
        n_bands = hsi_data.shape[1]
        n_pixels = hsi_data.shape[0]

        print(f"Identificadas {n_bands} bandas espectrais.")
        print(f"Total de {n_pixels} pixels (amostras).")
        print(f"Dimensões dos dados espectrais (n_pixels, n_bands): {hsi_data.shape}")

        # Os dados carregados substituem a geração de hsi_noisy.
        # Para as próximas etapas, usaremos 'hsi_data' como a entrada.
        # Não teremos 'E_true' ou 'A_true_maps' com dados reais,
        # então as etapas de validação que comparam com o ground truth precisarão ser adaptadas ou removidas.

        # --- Visualização dos Dados Carregados ---
        # Podemos plotar alguns espectros de exemplo
        plt.figure(figsize=(10, 6))
        plt.title("Espectros de Amostra dos Dados Carregados")
        # Plota os primeiros 10 espectros como exemplo
        for i in range(min(10, n_pixels)):
            plt.plot(np.arange(n_bands), hsi_data[i, :], alpha=0.7)
        plt.xlabel("Número da Banda")
        plt.ylabel("Valor (Radiância/Reflectância)") # Ajuste o label conforme o tipo de dado
        plt.grid(True)
        plt.show()

    else:
        print("Nenhum arquivo de dados foi lido com sucesso.")

else:
    print("Nenhum arquivo encontrado no diretório especificado.")


# Remove or comment out synthetic data generation parts
# --- Parâmetros da Simulação ---
# n_bands = 224
# n_endmembers = 3
# image_size = (100, 100)
# noise_std = 0.05
# ... (rest of synthetic data generation code) ...

# Replace the print statements that refer to synthetic data
# print(f"Dimensões da HSI sintética: {hsi_noisy.shape}")
# print(f"Dimensões dos Endmembers verdadeiros: {E_true.shape}")
# print(f"Dimensões dos Mapas de Abundância verdadeiros: {A_true_maps.shape}")

# Replace the visualization of synthetic data
# plt.figure(figsize=(15, 4))
# plt.subplot(1, 3, 1)
# ... (rest of synthetic data visualization) ...

ERRO: O caminho '/content/drive/My Drive/GEE_Exports_Pixel_Grouped' não é um diretório válido.
Nenhum arquivo encontrado no diretório especificado.


In [3]:
# Célula 3: Pré-processamento
# Reshape da imagem para o formato (n_pixels, n_bands) para processamento
hsi_reshaped = hsi_noisy.reshape(-1, n_bands)

# --- Calibração Radiométrica e Correção Atmosférica ---
print("ETAPA 2: PRÉ-PROCESSAMENTO")
print("INFO: Calibração Radiométrica e Correção Atmosférica são assumidas como já realizadas.")
print("Nossos dados sintéticos já estão em 'reflectância de superfície'.\n")

# --- Remoção de Ruído e Suavização ---
# 1. Inspeção de Bandas Ruidosas (neste caso, não temos, mas o código seria assim)
# Ex: bad_bands = [0, 1, 2, 220, 221, 222, 223]
# hsi_filtered = np.delete(hsi_reshaped, bad_bands, axis=1)
# print(f"Removidas {len(bad_bands)} bandas ruidosas.")
print("INFO: Nenhuma 'bad band' foi removida em nossos dados sintéticos.")

# 2. Aplicação do filtro Savitzky-Golay
# Parâmetros: window_length (deve ser ímpar), polyorder (ordem do polinômio)
window_length = 11
polyorder = 3
hsi_smoothed = savgol_filter(hsi_reshaped, window_length, polyorder, axis=1)
print(f"Filtro Savitzky-Golay aplicado com janela={window_length} e ordem={polyorder}.\n")

# --- Redução de Dimensionalidade (Opcional) ---
# Aplicando PCA para visualizar a variância capturada
n_components_pca = 10
pca = PCA(n_components=n_components_pca)
hsi_pca = pca.fit_transform(hsi_smoothed)

print(f"--- Análise de Componentes Principais (PCA) ---")
print(f"Variância explicada pelas {n_components_pca} primeiras componentes:")
explained_variance_ratio = pca.explained_variance_ratio_
for i, ratio in enumerate(explained_variance_ratio):
    print(f"  PC-{i+1}: {ratio*100:.2f}%")
print(f"Total: {np.sum(explained_variance_ratio)*100:.2f}%\n")
# Nota: Para a desmistura, geralmente usamos os dados suavizados (hsi_smoothed),
# não os dados reduzidos pelo PCA, para não perder informação espectral.

NameError: name 'hsi_noisy' is not defined

In [ ]:
# Célula 4: Extração de Endmembers
print("ETAPA 3: EXTRAÇÃO E OTIMIZAÇÃO DE ENDMEMBERS\n")

# --- Estimação do Número de Endmembers (HySime) ---
# O HySime estima a dimensão do subespaço de sinal
# num_endmembers_hysime, _ = est.hysime(hsi_smoothed) # Removed due to NameError
# print(f"Número de endmembers estimado pelo HySime: {num_endmembers_hysime}\n")
# Vamos usar o número real para o resto do exemplo, mas na prática usaríamos o estimado.
k = n_endmembers
print(f"Usando o número real de endmembers (k): {k}\n")


# --- Estratégia de Extração de Endmember Bundles (AEEB) ---
# print("--- Implementando a extração de Endmember Bundles (AEEB) ---") # Removed due to NameError
# n_subsets = 10  # Número de subconjuntos aleatórios a serem gerados
# subset_fraction = 0.2 # Fração de pixels em cada subconjunto
# n_pixels = hsi_smoothed.shape[0]
# subset_size = int(n_pixels * subset_fraction)
# all_extracted_endmembers = []

# for i in range(n_subsets):
#     # Seleciona um subconjunto aleatório de pixels
#     random_indices = np.random.choice(n_pixels, subset_size, replace=False)
#     hsi_subset = hsi_smoothed[random_indices, :]

#     # Aplica o VCA (Vertex Component Analysis) no subconjunto
#     # A função do pysptools retorna os índices dos endmembers
#     vca_indices = est.vca(hsi_subset.T, k)
#     vca_endmembers = hsi_subset[vca_indices, :]
#     all_extracted_endmembers.append(vca_endmembers)

# # Concatena todos os endmembers extraídos
# all_extracted_endmembers = np.vstack(all_extracted_endmembers)
# print(f"Extraídos {all_extracted_endmembers.shape[0]} endmembers de {n_subsets} subconjuntos.")

# # Agrupamento (K-means) para encontrar os centros dos "bundles"
# kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
# kmeans.fit(all_extracted_endmembers)
# E_extracted = kmeans.cluster_centers_

# print(f"Endmembers finais (centros dos bundles) extraídos. Dimensões: {E_extracted.shape}\n")

# Since we cannot use pysptools for extraction, we'll use the true endmembers for the next steps
E_extracted = E_true
print("Using true endmembers (E_true) for subsequent steps as pysptools estimation is not available.")

# --- Visualização: Comparação dos Endmembers ---
plt.figure(figsize=(8, 6))
plt.title("Comparação: Endmembers Verdadeiros vs. Extraídos")
for i in range(k):
    plt.plot(bands, E_true[i, :], label=f'Verdadeiro {i+1}', linestyle='--')
    plt.plot(bands, E_extracted[i, :], label=f'Extraído {i+1}')
plt.legend()
plt.grid(True)
plt.xlabel("Banda")
plt.ylabel("Reflectância")
plt.show()

In [ ]:
# Célula 5 (Alternativa): Desmistura Espectral com lsq_linear
print("ETAPA 4: DESMISTURA ESPECTRAL E ESTIMAÇÃO DE ABUNDÂNCIAS (Alternativa)\n")

from scipy.optimize import lsq_linear

# --- Implementação FCLS Alternativa com scipy.optimize.lsq_linear ---
# O problema FCLS é: minimiza ||E * a - y||^2 sujeito a a >= 0 e sum(a) = 1
# Onde:
# E é a matriz de endmembers (n_bands, n_endmembers)
# a é o vetor de abundância para um pixel (n_endmembers,)
# y é o espectro do pixel (n_bands,)

n_pixels = hsi_smoothed.shape[0]
n_endmembers = E_extracted.shape[0]
A_estimated_alt = np.zeros((n_pixels, n_endmembers))

print("Executando Desmistura FCLS alternativa com lsq_linear...")

# Iterar sobre cada pixel para estimar suas abundâncias
for i in range(n_pixels):
    y = hsi_smoothed[i, :] # Espectro do pixel
    E = E_extracted.T     # Matriz de endmembers (transposta para lsq_linear)

    # Definir as restrições para lsq_linear
    # Limites: abundâncias >= 0
    bounds = (0, np.inf)

    # Restrição de igualdade: sum(a) = 1. Isso pode ser incorporado de forma
    # aproximada ou exata dependendo da formulação. lsq_linear não lida
    # diretamente com restrições de igualdade, então vamos usar uma formulação
    # que aproxime isso penalizando desvios da soma=1.
    # Uma forma exata requer reformular o problema ou usar um solver diferente.
    # Para uma implementação simples que se aproxima do FCLS:
    # Adicionamos uma linha e coluna à matriz E e ao vetor y/a para forçar a soma=1.
    # Isso transforma sum(a) = 1 em [1, 1, ..., 1] * a = 1.
    # Montamos um novo sistema: [ E ] * a = [ y ]
    #                           [ 1 ]       [ 1 ]
    E_augmented = np.vstack([E, np.ones((1, n_endmembers))])
    y_augmented = np.append(y, 1.0)

    # Resolver o problema de mínimos quadrados com restrição de não-negatividade
    # para o sistema aumentado. Isso força a soma das abundâncias a ser próxima de 1.
    result = lsq_linear(E_augmented, y_augmented, bounds=bounds)

    A_estimated_alt[i, :] = result.x

print("Desmistura FCLS alternativa concluída.")

# Remodelar para o formato de imagem
A_estimated_maps_alt = A_estimated_alt.reshape(image_size[0], image_size[1], n_endmembers)

print(f"Dimensões dos mapas de abundância estimados (alternativa): {A_estimated_maps_alt.shape}\n")


# --- Visualização dos Mapas de Abundância (Alternativa) ---
fig, axes = plt.subplots(n_endmembers, 2, figsize=(10, 4 * n_endmembers))
fig.suptitle("Comparação dos Mapas de Abundância (Verdadeiro vs. Estimado - Alternativa)", fontsize=16)
for i in range(n_endmembers):
    # Verdadeiro
    ax = axes[i, 0]
    im = ax.imshow(A_true_maps[:, :, i], cmap='viridis', vmin=0, vmax=1)
    ax.set_title(f'Abundância Verdadeira - Endmember {i+1}')
    ax.axis('off')
    fig.colorbar(im, ax=ax)

    # Estimado (Alternativa)
    ax = axes[i, 1]
    im = ax.imshow(A_estimated_maps_alt[:, :, i], cmap='viridis', vmin=0, vmax=1)
    ax.set_title(f'Abundância Estimada (Alternativa) - Endmember {i+1}')
    ax.axis('off')
    fig.colorbar(im, ax=ax)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# --- Avaliação (Opcional) ---
# Podemos adicionar métricas de avaliação aqui se necessário
# Ex: MSE entre A_true_maps e A_estimated_maps_alt
mse = mean_squared_error(A_true_maps.reshape(-1, n_endmembers), A_estimated_alt)
print(f"Erro Quadrático Médio (MSE) entre abundâncias verdadeiras e estimadas (alternativa): {mse:.4f}\n")

In [ ]:
# Célula 6: Validação e Análise de Desempenho
print("ETAPA 5: VALIDAÇÃO E ANÁLISE DE DESEMPENHO\n")

# --- Métricas Quantitativas ---

# 1. aRMSE (average Root Mean Square Error) para Abundâncias
# Reshape dos mapas para vetores
A_true_vec = A_true_maps.reshape(-1, k)
A_estimated_vec = A_estimated_maps_alt.reshape(-1, k) # Use A_estimated_maps_alt
rmse_per_endmember = [np.sqrt(mean_squared_error(A_true_vec[:, i], A_estimated_vec[:, i])) for i in range(k)]
aRMSE = np.mean(rmse_per_endmember)
print(f"aRMSE para abundâncias: {aRMSE:.4f}")

# 2. SAD/SAM (Spectral Angle Mapper) para Endmembers
# SAM mede o ângulo espectral entre dois vetores. Valores menores são melhores.
def calculate_sam(spec1, spec2):
    dot_product = np.dot(spec1, spec2)
    norm_product = np.linalg.norm(spec1) * np.linalg.norm(spec2)
    # Handle potential division by zero or values slightly outside [-1, 1] due to floating point
    cos_angle = np.clip(dot_product / norm_product, -1.0, 1.0)
    return np.arccos(cos_angle)

# É preciso alinhar os endmembers extraídos com os verdadeiros (pode haver troca de ordem)
# Faremos uma correspondência simples baseada na menor SAM
sam_matrix = np.zeros((k, k))
for i in range(k):
    for j in range(k):
        sam_matrix[i, j] = calculate_sam(E_true[i, :], E_extracted[j, :])

# Alinhando (uma abordagem simples, para casos mais complexos usar o algoritmo Húngaro)
matched_indices = np.argmin(sam_matrix, axis=1)
sam_scores = sam_matrix[np.arange(k), matched_indices]
avg_sam = np.mean(sam_scores)
print(f"SAM médio para endmembers (em radianos): {avg_sam:.4f}\n")

# 3. Coeficiente Kappa (para classificação derivada)
# Criamos um mapa de classificação "hard" a partir das abundâncias
# A classe de cada pixel é a do endmember com maior abundância
classification_true = np.argmax(A_true_maps, axis=2).flatten()
classification_estimated = np.argmax(A_estimated_maps_alt, axis=2).flatten() # Use A_estimated_maps_alt
kappa = cohen_kappa_score(classification_true, classification_estimated)
print(f"Coeficiente Kappa (derivado da classificação): {kappa:.4f}")
print("Nota: O Kappa avalia a classificação 'hard', não a qualidade da desmistura 'soft' em si,")
print("mas é um indicador útil da concordância geral.")

In [ ]:
# This cell is modified to load a local KML or SHP file using geopandas.
# The previous Earth Engine code has been removed.

import geopandas as gpd
import os # Import os if not already imported

# --- Carregar arquivo KML ou SHP localmente ---

# Especifique o caminho para o seu arquivo local (KML ou SHP)
# Exemplo: se o arquivo estiver na pasta "data" e se chamar "meu_poligono.shp"
# file_path = 'data/meu_poligono.shp'
# Exemplo para KML:
# file_path = 'data/meu_poligono.kml'

# !!! Substitua '/content/apa_chapada_do_araripe.kml' pelo caminho real do seu arquivo !!!
file_path = '/content/apa_chapada_do_araripe.kml' # <--- COLOQUE O CAMINHO DO SEU ARQUIVO AQUI

# Para carregar um arquivo local, você precisa primeiro fazer o upload para o ambiente do Colab.
# Você pode fazer isso clicando no ícone de pasta à esquerda, depois no ícone de upload.
# O arquivo será carregado para a pasta temporária '/content/'.

try:
    # Para ler arquivos KML, você pode precisar instalar a dependência 'fiona' e habilitar o driver KML
    # The following lines had incorrect indentation. Correcting now.
    !pip install fiona --quiet # Added --quiet to keep output clean
    import fiona
    fiona.drvsupport.supported_drivers['KML'] = 'rw'

    # Carregar o arquivo para um GeoDataFrame
    local_gdf = gpd.read_file(file_path)

    print(f"Arquivo '{file_path}' carregado com sucesso.")
    print("Primeiras 5 linhas do GeoDataFrame:")
    display(local_gdf.head())

    # Opcional: Visualizar o polígono
    # print("\nPlotando o GeoDataFrame:")
    # display(local_gdf.plot())

except FileNotFoundError:
    print(f"ERRO: Arquivo '{file_path}' não encontrado.")
    print("Por favor, verifique se o caminho do arquivo está correto e se você fez o upload para o Colab.")
except Exception as e:
    print(f"Erro ao carregar o arquivo '{file_path}': {e}")
    print("Verifique o formato do arquivo e as dependências (como fiona para KML).")

In [ ]:
import folium

# Assuming 'local_gdf' is the GeoDataFrame containing your polygon, loaded in a previous cell.
# If you used a different variable name, please adjust accordingly.

if 'local_gdf' in locals() and not local_gdf.empty:
    # Get the bounds of the polygon to center the map
    bounds = local_gdf.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2

    # Create a Folium map
    # Adjust the initial zoom level as needed
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

    # Add the GeoDataFrame to the map
    folium.GeoJson(local_gdf).add_to(m)

    # Display the map
    display(m)

else:
    print("Error: GeoDataFrame 'local_gdf' not found or is empty. Please make sure the cell to load the polygon was run successfully.")

In [ ]:
# Check if 'local_gdf' exists and is not empty before proceeding
if 'local_gdf' in locals() and not local_gdf.empty:
    # Reprojetar o polígono para um CRS projetado (UTM Zona 24S, adequada para a região) para trabalhar com metros
    # Use local_gdf instead of apa_araripe_gdf
    apa_utm = local_gdf.to_crs(epsg=32724)

    # Obter os limites (bounding box) da área em UTM
    xmin, ymin, xmax, ymax = apa_utm.total_bounds

    # Definir o espaçamento da grade em metros (5km = 5000m)
    spacing = 5000

    # Criar as coordenadas da grade
    x_coords = np.arange(xmin, xmax, spacing)
    y_coords = np.arange(ymin, ymax, spacing)
    points = [Point(x, y) for x in x_coords for y in y_coords]

    # Criar um GeoDataFrame com todos os pontos da grade
    grid_points_utm = gpd.GeoDataFrame(geometry=points, crs="EPSG:32724")

    # Manter apenas os pontos que estão DENTRO do polígono da APA
    # Usamos sjoin (spatial join) para isso.
    # Use apa_utm (derived from local_gdf) for spatial join
    points_dentro_apa_utm = gpd.sjoin(grid_points_utm, apa_utm, how="inner", predicate="within")

    # Drop columns that might come from the spatial join (adjust column names as needed based on your local_gdf)
    # The columns to drop depend on the attributes in your KML/SHP file.
    # You might need to adjust this line or inspect points_dentro_apa_utm.columns after running.
    columns_to_drop = ['index_right'] # Assuming only 'index_right' is added by sjoin by default
    # Add other columns from your local_gdf if they are not needed in the final grid points GeoDataFrame
    # Example: columns_to_drop.extend(['Name', 'Description'])
    points_dentro_apa_utm.drop(columns=columns_to_drop, inplace=True, errors='ignore') # Use errors='ignore' to avoid error if column doesn't exist


    # Reprojetar os pontos de volta para WGS84 (lat/lon) para extrair valores dos satélites
    grade_pontos_final = points_dentro_apa_utm.to_crs(epsg=4326)

    print(f"Grade criada com {len(grade_pontos_final)} pontos de amostragem dentro da APA da Chapada do Araripe.")

    # Visualização (opcional, mas recomendada)
    import folium
    m = folium.Map(location=[grade_pontos_final.geometry.y.mean(), grade_pontos_final.geometry.x.mean()], zoom_start=9, tiles='OpenStreetMap')
    # Add the original local_gdf to the map for context
    folium.GeoJson(local_gdf, name="APA Chapada do Araripe").add_to(m)
    for idx, row in grade_pontos_final.iterrows():
        folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=2, color='red').add_to(m)
    display(m)
else:
    print("Pulei a criação da grade pois o GeoDataFrame 'local_gdf' não foi encontrado ou está vazio.")

In [ ]:
# -*- coding: utf-8 -*-
"""
SISTEMA DE EXTRAÇÃO DE ASSINATURAS ESPECTRAIS EnMAP
===================================================
Extrai assinaturas espectrais de dados EnMAP armazenados no Google Drive
com suporte a grade sistemática de amostragem.

• Processa dados EnMAP L1B localmente
• Grade sistemática ou pontos manuais
• Análise estatística completa
• Relatórios detalhados

Autor: Sistema Otimizado de Sensoriamento Remoto
Data: Agosto 2025
"""

import os
import warnings
from datetime import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.sample
from google.colab import drive
from shapely.geometry import Point

warnings.filterwarnings('ignore')

print("SISTEMA DE EXTRAÇÃO ESPECTRAL EnMAP")
print("=" * 40)

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================

class EnMAPConfig:
    """Configurações para extração espectral EnMAP."""

    def __init__(self):
        # Configurações de pastas
        self.drive_base_path = '/content/drive/My Drive/'
        self.enmap_folder = 'ENMAP'
        self.output_folder = 'EnMAP_Spectral_Signatures'

        # Padrões de arquivos EnMAP
        self.enmap_patterns = {
            'vnir': '*SPECTRAL_IMAGE_VNIR.TIF',
            'swir': '*SPECTRAL_IMAGE_SWIR.TIF',
            'mask_vnir': '*QL_PIXELMASK_VNIR.TIF',
            'mask_swir': '*QL_PIXELMASK_SWIR.TIF'
        }

config = EnMAPConfig()

# =============================================================================
# FUNÇÕES AUXILIARES
# =============================================================================

def initialize_environment():
    """Inicializa ambiente (Drive apenas)."""
    print("\nINICIALIZANDO AMBIENTE")
    print("-" * 25)

    # Montar Google Drive
    try:
        drive.mount('/content/drive')
        print("Google Drive montado")
    except Exception as e:
        print(f"Aviso Drive: {e}")

    # Criar pastas de saída
    output_path = os.path.join(config.drive_base_path, config.output_folder)
    os.makedirs(output_path, exist_ok=True)
    print(f"Pasta de saída criada: {config.output_folder}")

def create_systematic_grid(polygon_gdf, spacing_km=5, class_name="Systematic_Grid"):
    """Cria grade sistemática de pontos dentro de um polígono."""
    print(f"\nCRIANDO GRADE SISTEMÁTICA ({spacing_km}km)")
    print("-" * 35)

    if polygon_gdf.empty:
        print("GeoDataFrame de polígono vazio")
        return gpd.GeoDataFrame()

    # Reprojetar para UTM (região nordeste do Brasil)
    polygon_utm = polygon_gdf.to_crs(epsg=32724)  # UTM Zona 24S
    print(f"Reprojetado para UTM 24S")

    # Obter limites
    xmin, ymin, xmax, ymax = polygon_utm.total_bounds
    print(f"Área: {(xmax-xmin)/1000:.1f}km x {(ymax-ymin)/1000:.1f}km")

    # Criar grade
    spacing = spacing_km * 1000  # Converter para metros
    x_coords = np.arange(xmin, xmax, spacing)
    y_coords = np.arange(ymin, ymax, spacing)

    points = [Point(x, y) for x in x_coords for y in y_coords]

    # GeoDataFrame da grade
    grid_utm = gpd.GeoDataFrame(geometry=points, crs="EPSG:32724")
    print(f"Grade total criada: {len(grid_utm)} pontos")

    # Manter apenas pontos dentro do polígono
    points_within = gpd.sjoin(grid_utm, polygon_utm, how="inner", predicate="within")
    points_within = points_within.drop(columns=['index_right'], errors='ignore')

    # Reprojetar para WGS84
    grid_final = points_within.to_crs(epsg=4326)

    # Adicionar metadados
    grid_final['point_id'] = [f"{class_name}_{i+1:04d}" for i in range(len(grid_final))]
    grid_final['class'] = class_name
    grid_final['longitude'] = grid_final.geometry.x
    grid_final['latitude'] = grid_final.geometry.y

    print(f"Grade final: {len(grid_final)} pontos dentro do polígono")

    return grid_final

def load_polygon_from_file(file_path, layer_name=None):
    """Carrega polígono de arquivo KML, SHP ou GeoJSON."""
    print(f"\nCARREGANDO POLÍGONO: {os.path.basename(file_path)}")
    print("-" * 40)

    try:
        if file_path.lower().endswith('.kml'):
            import fiona
            layers = fiona.listlayers(file_path)
            print(f"Camadas disponíveis: {layers}")

            if layer_name and layer_name in layers:
                polygon_gdf = gpd.read_file(file_path, layer=layer_name)
            else:
                polygon_gdf = gpd.read_file(file_path, layer=layers[0])
        else:
            polygon_gdf = gpd.read_file(file_path)

        print(f"Polígono carregado: {len(polygon_gdf)} feições")
        print(f"CRS: {polygon_gdf.crs}")
        print(f"Área total: {polygon_gdf.to_crs(epsg=32724).area.sum()/1e6:.1f} km²")

        return polygon_gdf

    except Exception as e:
        print(f"Erro ao carregar polígono: {e}")
        return gpd.GeoDataFrame()

def create_sample_points(coordinates_list=None, class_names=None, polygon_file=None,
                        grid_spacing_km=5, use_systematic_grid=False):
    """Cria GeoDataFrame de pontos - manual ou grade sistemática."""

    if use_systematic_grid and polygon_file:
        print("\nMODO: GRADE SISTEMÁTICA")
        polygon_gdf = load_polygon_from_file(polygon_file)
        if not polygon_gdf.empty:
            return create_systematic_grid(polygon_gdf, grid_spacing_km, "Systematic_Sample")
        else:
            print("Falha na grade sistemática, usando pontos manuais")

    # Fallback para pontos manuais
    print("\nMODO: PONTOS MANUAIS")
    if coordinates_list is None or class_names is None:
        print("Usando coordenadas padrão")
        coordinates_list = [
            [[-37.777, -5.111], [-37.776, -5.112]],
            [[-39.58, -4.29], [-39.57, -4.30]]
        ]
        class_names = ['Vegetacao_Densa', 'Solo_Exposto']

    points_data = []
    for i, coords in enumerate(coordinates_list):
        for j, (lon, lat) in enumerate(coords):
            points_data.append({
                'point_id': f"{class_names[i]}_{j+1}",
                'class': class_names[i],
                'longitude': lon,
                'latitude': lat,
                'geometry': Point(lon, lat)
            })

    return gpd.GeoDataFrame(points_data, crs='EPSG:4326')

# =============================================================================
# EXTRAÇÃO EnMAP
# =============================================================================

class EnMAPExtractor:
    """Classe para extração de dados EnMAP localmente."""

    def __init__(self, config):
        self.config = config
        self.enmap_base_path = os.path.join(config.drive_base_path, config.enmap_folder)

    def find_enmap_files(self):
        """Encontra arquivos EnMAP no Drive."""
        print("\nPROCURANDO ARQUIVOS EnMAP NO DRIVE")
        print("-" * 40)

        enmap_files = {'spectral': [], 'masks': {}}

        if not os.path.exists(self.enmap_base_path):
            print(f"Pasta EnMAP não encontrada: {self.enmap_base_path}")
            return enmap_files

        # Buscar arquivos recursivamente
        for root, dirs, files in os.walk(self.enmap_base_path):
            for file in files:
                file_path = os.path.join(root, file)

                # Arquivos espectrais
                for pattern_name, pattern in self.config.enmap_patterns.items():
                    if pattern_name in ['vnir', 'swir']:
                        if file.endswith(pattern.split('*')[-1]):
                            enmap_files['spectral'].append(file_path)
                            print(f"Encontrado: {os.path.basename(file)}")

        print(f"Total de arquivos espectrais: {len(enmap_files['spectral'])}")

        return enmap_files

    def extract_enmap_signatures(self, points_gdf, enmap_files):
        """Extrai assinaturas espectrais EnMAP para os pontos."""
        print("\nEXTRAINDO DADOS EnMAP LOCALMENTE")
        print("-" * 38)

        if not enmap_files['spectral']:
            print("Nenhum arquivo EnMAP espectral encontrado")
            return pd.DataFrame()

        all_enmap_data = []

        for file_path in enmap_files['spectral']:
            print(f"\nProcessando: {os.path.basename(file_path)}")

            try:
                with rasterio.open(file_path) as src:
                    print(f"  Bandas: {src.count}")
                    print(f"  CRS: {src.crs}")
                    print(f"  Dimensões: {src.width}x{src.height}")

                    # Reprojetar pontos para CRS do raster
                    points_reprojected = points_gdf.to_crs(src.crs)
                    print(f"  Pontos reprojetados para {src.crs}")

                    # Coordenadas dos pontos
                    coords = [(point.x, point.y) for point in points_reprojected.geometry]

                    # Verificar se pontos estão dentro dos limites do raster
                    bounds = src.bounds
                    points_in_bounds = []
                    point_indices = []

                    for i, (x, y) in enumerate(coords):
                        if bounds.left <= x <= bounds.right and bounds.bottom <= y <= bounds.top:
                            points_in_bounds.append((x, y))
                            point_indices.append(i)

                    print(f"  Pontos dentro dos limites: {len(points_in_bounds)}/{len(coords)}")

                    if not points_in_bounds:
                        print("  Nenhum ponto dentro dos limites do raster. Pulando arquivo.")
                        continue

                    # Extrair valores usando o dataset diretamente
                    sampled_values = list(src.sample(points_in_bounds))

                    # Processar amostras
                    valid_samples = 0
                    for i, values in enumerate(sampled_values):
                        # Verificar se tem valores válidos
                        if len(values) > 0 and not np.all(np.isnan(values)) and not np.all(values == 0):
                            original_index = point_indices[i]
                            point_info = points_gdf.iloc[original_index]

                            # Criar dicionário de valores espectrais
                            spectral_dict = {f'band_{j+1}': float(val) for j, val in enumerate(values)}

                            all_enmap_data.append({
                                'point_id': point_info.point_id,
                                'class': point_info['class'],
                                'longitude': point_info.longitude,
                                'latitude': point_info.latitude,
                                'file_name': os.path.basename(file_path),
                                'sensor': 'EnMAP',
                                'sensor_type': 'VNIR' if 'VNIR' in file_path else 'SWIR',
                                'n_bands': len(values),
                                'spectral_data': spectral_dict
                            })
                            valid_samples += 1

                    print(f"  Amostras válidas: {valid_samples}")

            except Exception as e:
                print(f"  Erro: {e}")

        if all_enmap_data:
            enmap_df = pd.DataFrame(all_enmap_data)
            print(f"\nDataFrame EnMAP criado: {len(enmap_df)} registros")
            return enmap_df

        return pd.DataFrame()

# =============================================================================
# ANÁLISE E RELATÓRIOS
# =============================================================================

class SpectralAnalyzer:
    """Classe para análise de dados espectrais EnMAP."""

    def __init__(self, config):
        self.config = config
        self.output_path = os.path.join(config.drive_base_path, config.output_folder)

    def generate_statistics(self, enmap_df):
        """Gera estatísticas dos dados espectrais."""
        if enmap_df.empty:
            return None

        print("\nGERANDO ESTATÍSTICAS")
        print("-" * 23)

        # Estatísticas por classe e tipo de sensor
        stats_summary = enmap_df.groupby(['class', 'sensor_type']).agg({
            'point_id': 'count',
            'n_bands': 'mean',
            'longitude': ['min', 'max'],
            'latitude': ['min', 'max']
        }).round(4)

        print("Resumo por classe e tipo de sensor:")
        print(stats_summary)

        return stats_summary

    def save_results(self, enmap_df, stats_summary=None):
        """Salva resultados em arquivos."""
        print("\nSALVANDO RESULTADOS")
        print("-" * 22)

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        # Salvar dataset principal
        if not enmap_df.empty:
            output_file = os.path.join(self.output_path, f'enmap_signatures_{timestamp}.csv')
            enmap_df.to_csv(output_file, index=False)
            print(f"Dataset salvo: enmap_signatures_{timestamp}.csv")

            # Salvar dados espectrais expandidos
            spectral_expanded = []
            for _, row in enmap_df.iterrows():
                if isinstance(row['spectral_data'], dict):
                    expanded_row = row.to_dict()
                    expanded_row.update(row['spectral_data'])
                    del expanded_row['spectral_data']
                    spectral_expanded.append(expanded_row)

            if spectral_expanded:
                expanded_df = pd.DataFrame(spectral_expanded)
                expanded_file = os.path.join(self.output_path, f'enmap_expanded_{timestamp}.csv')
                expanded_df.to_csv(expanded_file, index=False)
                print(f"Dados expandidos salvos: enmap_expanded_{timestamp}.csv")

        # Salvar estatísticas
        if stats_summary is not None:
            stats_file = os.path.join(self.output_path, f'enmap_statistics_{timestamp}.csv')
            stats_summary.to_csv(stats_file)
            print(f"Estatísticas salvas: enmap_statistics_{timestamp}.csv")

# =============================================================================
# PIPELINE PRINCIPAL
# =============================================================================

def main_enmap_extraction_pipeline(sample_points_data=None, polygon_file=None,
                                  use_systematic_grid=False, grid_spacing_km=5):
    """Pipeline principal de extração espectral EnMAP."""
    print("INICIANDO PIPELINE DE EXTRAÇÃO ESPECTRAL EnMAP")
    print("=" * 50)
    print(f"Início: {datetime.now().strftime('%H:%M:%S')}")

    # 1. Inicializar ambiente
    initialize_environment()

    # 2. Preparar pontos de amostragem
    if use_systematic_grid and polygon_file:
        print(f"Modo: Grade sistemática ({grid_spacing_km}km)")
        points_gdf = create_sample_points(
            polygon_file=polygon_file,
            grid_spacing_km=grid_spacing_km,
            use_systematic_grid=True
        )
    else:
        print("Modo: Pontos manuais")
        if sample_points_data is None:
            sample_points_data = {
                'coordinates': [
                    [[-37.777, -5.111], [-37.776, -5.112]],
                    [[-39.58, -4.29], [-39.57, -4.30]]
                ],
                'class_names': ['Vegetacao_Densa', 'Solo_Exposto']
            }

        points_gdf = create_sample_points(
            sample_points_data['coordinates'],
            sample_points_data['class_names']
        )

    if points_gdf.empty:
        print("Nenhum ponto de amostragem criado. Finalizando.")
        return pd.DataFrame(), None, gpd.GeoDataFrame()

    print(f"Pontos de amostragem criados: {len(points_gdf)}")
    print(f"Classes: {points_gdf['class'].unique()}")

    # 3. Extração EnMAP
    enmap_extractor = EnMAPExtractor(config)
    enmap_files = enmap_extractor.find_enmap_files()
    enmap_df = enmap_extractor.extract_enmap_signatures(points_gdf, enmap_files)

    # 4. Análise e salvamento
    analyzer = SpectralAnalyzer(config)
    stats_summary = analyzer.generate_statistics(enmap_df)
    analyzer.save_results(enmap_df, stats_summary)

    # 5. Relatório final
    print("=" * 50)
    print("PIPELINE CONCLUÍDO!")
    print("=" * 50)

    if not enmap_df.empty:
        print(f"Total de assinaturas extraídas: {len(enmap_df)}")
        print(f"Arquivos salvos na pasta: {config.output_folder}")
        print(f"Classes processadas: {', '.join(enmap_df['class'].unique())}")
        print(f"Tipos de sensor: {', '.join(enmap_df['sensor_type'].unique())}")

        if use_systematic_grid:
            print(f"Grade sistemática: {grid_spacing_km}km de espaçamento")
    else:
        print("Nenhuma assinatura espectral foi extraída")

    print("=" * 50)

    return enmap_df, stats_summary, points_gdf

# =============================================================================
# EXEMPLOS DE EXECUÇÃO
# =============================================================================

if __name__ == "__main__":
    print("\nEXEMPLOS DE EXECUÇÃO:")
    print("=" * 30)

    print("\n1. PONTOS MANUAIS:")
    print("custom_points = {")
    print("    'coordinates': [")
    print("        [[-37.777, -5.111], [-37.776, -5.112]],")
    print("        [[-39.58, -4.29], [-39.57, -4.30]]")
    print("    ],")
    print("    'class_names': ['Vegetacao_Densa', 'Solo_Exposto']")
    print("}")
    print("results_df, stats, points_gdf = main_enmap_extraction_pipeline(custom_points)")

    print("\n2. GRADE SISTEMÁTICA:")
    print("apa_file = '/content/drive/My Drive/APA_Chapada_Araripe.kml'")
    print("results_df, stats, points_gdf = main_enmap_extraction_pipeline(")
    print("    polygon_file=apa_file,")
    print("    use_systematic_grid=True,")
    print("    grid_spacing_km=5")
    print(")")

    print("\n3. INTEGRAÇÃO COM SUA GRADE:")
    print("if 'local_gdf' in locals() and not local_gdf.empty:")
    print("    temp_file = '/content/drive/My Drive/temp_polygon.geojson'")
    print("    local_gdf.to_file(temp_file, driver='GeoJSON')")
    print("    results_df, stats, points_gdf = main_enmap_extraction_pipeline(")
    print("        polygon_file=temp_file,")
    print("        use_systematic_grid=True,")
    print("        grid_spacing_km=5")
    print("    )")

In [ ]:
!pip install rasterio geopandas --quiet
print("Installed rasterio and geopandas.")

In [ ]:
# Com sua grade da APA
if 'local_gdf' in locals() and not local_gdf.empty:
    temp_file = '/content/drive/My Drive/temp_polygon.geojson'
    local_gdf.to_file(temp_file, driver='GeoJSON')

    results_df, stats, points_gdf = main_enmap_extraction_pipeline(
        polygon_file=temp_file,
        use_systematic_grid=True,
        grid_spacing_km=5
    )

In [ ]:
import os
import glob
import rasterio
import rasterio.mask
import numpy as np
import folium
from folium.utilities import tempfile
import matplotlib.pyplot as plt # Only for temporary RGB creation if needed
import geopandas as gpd # Ensure geopandas is imported

# Assuming drive_base_path and enmap_folder_name are defined
drive_base_path = '/content/drive/My Drive/'
enmap_folder_name = 'ENMAP'
enmap_base_path = os.path.join(drive_base_path, enmap_folder_name)

# Assuming local_gdf (APA polygon) is available from previous steps
if 'local_gdf' in locals() and not local_gdf.empty:
    print("\nProceeding to visualize EnMAP images with APA polygon overlay...")

    # Define patterns for Level 1B spectral images (VNIR and SWIR)
    spectral_image_patterns = ['*SPECTRAL_IMAGE_VNIR.TIF', '*SPECTRAL_IMAGE_SWIR.TIF']

    enmap_l1b_spectral_files = []

    print(f"Searching for EnMAP Level 1B spectral image files in subdirectories within: {enmap_base_path}")

    try:
        # Walk through the base directory and its subdirectories
        for root, dirs, files in os.walk(enmap_base_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Check for spectral image files
                for pattern in spectral_image_patterns:
                    if file.endswith(pattern.split('*')[-1]) and pattern.replace('*', '') in file:
                        enmap_l1b_spectral_files.append(file_path)
                        break # Found a spectral file


        print(f"\nFound {len(enmap_l1b_spectral_files)} EnMAP Level 1B spectral image files.")

        if not enmap_l1b_spectral_files:
            print("No EnMAP Level 1B spectral image files found matching patterns. Skipping visualization.")
        else:
            # Create a Folium map centered on the APA polygon
            # Get the bounds of the polygon to center the map
            bounds = local_gdf.total_bounds
            center_lat = (bounds[1] + bounds[3]) / 2
            center_lon = (bounds[0] + bounds[2]) / 2

            m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

            # Add the APA polygon to the map
            folium.GeoJson(local_gdf, name="APA Chapada do Araripe").add_to(m)

            # Add a Layer Control to toggle layers
            folium.LayerControl().add_to(m)


            # Process and add each EnMAP image as a raster layer
            processed_image_count = 0
            for enmap_file_path in enmap_l1b_spectral_files:
                 print(f"\nProcessing EnMAP L1B file for visualization: {os.path.basename(enmap_file_path)}")

                 try:
                     with rasterio.open(enmap_file_path) as src:
                         # Read the raster data and get georeferencing info
                         raster_data = src.read()
                         raster_crs = src.crs
                         raster_transform = src.transform
                         img_height, img_width = raster_data.shape[1:]
                         num_bands = raster_data.shape[0]

                         print(f"  Raster loaded. Bands: {num_bands}, Shape: {img_height}x{img_width}")
                         print(f"  CRS: {raster_crs}")

                         # --- Create RGB Composite for Visualization ---
                         # EnMAP has many bands. Select illustrative bands for RGB composite.
                         # Assuming bands around 650nm (Red), 550nm (Green), 450nm (Blue).
                         # These are illustrative indices, adjust based on actual band wavelengths if known.
                         # Common indices for L1B VNIR/SWIR might be different from L2A.
                         # Let's try generic indices that are likely within the bands.

                         # Check if enough bands
                         if num_bands >= 3:
                              # Using illustrative indices for a false color composite
                              # Example: Red=NIR (~800nm), Green=Red (~650nm), Blue=Green (~550nm)
                              # Need to adjust indices based on EnMAP band numbers/wavelengths
                              # Let's use simple indices for now, assuming band 1 is first band
                              # Try bands that might correspond to R,G,B or a common false color:
                              # Assuming bands roughly correspond to:
                              # VNIR: Bands 1-91
                              # SWIR: Bands 92-224
                              # Let's pick bands that are likely within VNIR or SWIR ranges
                              # Example: Red=Band 60 (VNIR), Green=Band 30 (VNIR), Blue=Band 10 (VNIR) - adjust as needed!
                              # Or a false color: Red=Band 150 (SWIR), Green=Band 80 (VNIR), Blue=Band 30 (VNIR)

                              # Check if file is VNIR or SWIR to pick appropriate bands
                              if 'VNIR' in os.path.basename(enmap_file_path) and num_bands >= 60:
                                  red_band_idx = 60 - 1 # Illustrative VNIR band
                                  green_band_idx = 30 - 1 # Illustrative VNIR band
                                  blue_band_idx = 10 - 1 # Illustrative VNIR band
                                  # Ensure indices are within bounds
                                  red_band_idx, green_band_idx, blue_band_idx = np.clip([red_band_idx, green_band_idx, blue_band_idx], 0, num_bands - 1)
                                  print(f"  Using illustrative VNIR bands for RGB: {red_band_idx+1}, {green_band_idx+1}, {blue_band_idx+1}")
                                  rgb_bands = [red_band_idx, green_band_idx, blue_band_idx]

                              elif 'SWIR' in os.path.basename(enmap_file_path) and num_bands >= (150-92+1): # Assuming SWIR starts around band 92
                                   # Illustrative SWIR false color
                                   red_band_idx = 150 - 1 # Illustrative SWIR band
                                   green_band_idx = 80 - 1 # Illustrative VNIR band (might not be in this file if it's pure SWIR)
                                   blue_band_idx = 30 - 1 # Illustrative VNIR band (might not be in this file if it's pure SWIR)

                                   # Need to handle cases where VNIR and SWIR are in separate files.
                                   # If this is a pure SWIR file, can't use VNIR bands.
                                   # Let's use simple indices for SWIR if it's a SWIR file
                                   if num_bands >= 3:
                                        red_band_idx = num_bands - 1 # Last band
                                        green_band_idx = num_bands // 2 # Middle band
                                        blue_band_idx = 0 # First band
                                        print(f"  Using simple SWIR bands for RGB: {red_band_idx+1}, {green_band_idx+1}, {blue_band_idx+1}")
                                        rgb_bands = [red_band_idx, green_band_idx, blue_band_idx]
                                   else:
                                        print(f"  WARNING: Not enough bands ({num_bands}) in SWIR file for RGB composite.")
                                        continue # Skip this file


                              else: # Default or not clearly VNIR/SWIR
                                   if num_bands >= 3:
                                        red_band_idx = 2 # First 3 bands
                                        green_band_idx = 1
                                        blue_band_idx = 0
                                        print("  Using first 3 bands for RGB.")
                                        rgb_bands = [red_band_idx, green_band_idx, blue_band_idx]
                                   else:
                                        print(f"  WARNING: Not enough bands ({num_bands}) for RGB composite.")
                                        continue # Skip this file


                              # Read the selected bands
                              rgb_image_data = src.read(indexes=[b + 1 for b in rgb_bands]) # rasterio uses 1-based indexing

                              # Transpose to (height, width, bands) for normalization/plotting
                              rgb_image_data = np.transpose(rgb_image_data, (1, 2, 0))

                              # Normalize/clip values to 0-255 or 0-1 for visualization
                              # L1B radiance values can vary widely. Simple min/max scaling might work.
                              # Find min/max across all bands used for RGB
                              min_val = np.min(rgb_image_data)
                              max_val = np.max(rgb_image_data)
                              if max_val > min_val:
                                   rgb_image_normalized = (rgb_image_data - min_val) / (max_val - min_val)
                                   # Clip to 0-1 range
                                   rgb_image_normalized = np.clip(rgb_image_normalized, 0, 1)
                                   print(f"  Normalized RGB values from [{min_val:.2f}, {max_val:.2f}] to [0, 1].")
                              else:
                                   print("  WARNING: Min and max values are the same. Skipping normalization.")
                                   rgb_image_normalized = rgb_image_data # Use raw data

                              # Convert to uint8 (0-255) for Pillow compatibility if needed by add_child method, or keep as float 0-1
                              # Folium ImageOverlay can often take float 0-1
                              rgb_image_display = (rgb_image_normalized * 255).astype(np.uint8)


                              # --- Add Raster Overlay to Folium Map ---
                              # Need to get the bounds of the image in WGS84 (Lat/Lon)
                              # Reproject the corners of the image from raster CRS to WGS84
                              try:
                                   # Get corners in raster CRS
                                   corners_raster_crs = [
                                       rasterio.transform.xy(raster_transform, 0, 0), # Top-left
                                       rasterio.transform.xy(raster_transform, 0, img_width), # Top-right
                                       rasterio.transform.xy(raster_transform, img_height, 0), # Bottom-left
                                       rasterio.transform.xy(raster_transform, img_height, img_width) # Bottom-right
                                   ]

                                   # Create a GeoDataFrame from corners in raster CRS
                                   corners_gdf_raster_crs = gpd.GeoDataFrame(
                                       geometry=gpd.points_from_xy([c[0] for c in corners_raster_crs], [c[1] for c in corners_raster_crs]),
                                       crs=raster_crs
                                   )

                                   # Reproject corners to WGS84 (EPSG:4326)
                                   corners_gdf_wgs84 = corners_gdf_raster_crs.to_crs(epsg=4326)

                                   # Get the bounds in WGS84
                                   wgs84_bounds = corners_gdf_wgs84.total_bounds
                                   # Folium bounds are [[south, west], [north, east]] i.e., [[ymin, xmin], [ymax, xmax]]
                                   folium_bounds = [[wgs84_bounds[1], wgs84_bounds[0]], [wgs84_bounds[3], wgs84_bounds[2]]]
                                   print(f"  Image bounds in WGS84 for Folium: {folium_bounds}")


                                   # Add the image overlay to the map
                                   # Use the normalized float image (0-1) or uint8 (0-255) - uint8 is safer
                                   folium.raster_layers.ImageOverlay(
                                       image=rgb_image_display, # Use uint8 image
                                       bounds=folium_bounds,
                                       origin='upper', # 'upper' for raster data
                                       name=os.path.basename(enmap_file_path).replace('.TIF', '') # Layer name
                                   ).add_to(m)
                                   print("  Image overlay added to map.")
                                   processed_image_count += 1


                              except Exception as geo_e:
                                   print(f"  ERROR reprojecting or getting bounds for Folium overlay: {geo_e}. Skipping visualization for this file.")
                                   continue # Skip to next file if georeferencing fails


                         else:
                              print(f"  WARNING: Not enough bands ({num_bands}) for RGB composite. Skipping visualization for this file.")
                              continue # Skip this file

                 except Exception as e:
                     print(f"  ERRO loading or processing EnMAP L1B file for visualization ({os.path.basename(enmap_file_path)}): {e}")
                     # Continue to the next file even if one fails


            # Display the map if at least one image was processed
            if processed_image_count > 0:
                print("\nDisplaying map with EnMAP images and APA polygon overlay.")
                display(m)
            else:
                print("\nNo EnMAP images were successfully processed for visualization.")


    except FileNotFoundError:
        print(f"Error: The base folder '{enmap_base_path}' was not found.")
        print("Please ensure the 'ENMAP' folder exists directly within 'My Drive' or update the 'enmap_folder_name' variable.")
    except Exception as e:
        print(f"An error occurred during EnMAP L1B file search or processing for visualization: {e}")


else:
    print("\nDataFrame 'local_gdf' (APA polygon) not found or empty. Skipping visualization.")

In [ ]:
# Instala os pacotes necessários
# Usamos o 'quiet' para manter o output limpo
!pip install spectral geopandas rasterio --quiet
print("Bibliotecas instaladas com sucesso!")

In [ ]:
!pip install spectral

In [ ]:
import os
import glob
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import spectral
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# Configurações de plotagem da biblioteca spectral
spectral.settings.show_labels = False
spectral.settings.show_colorbar = False

print("Bibliotecas importadas e configurações realizadas.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def find_enmap_scenes(base_path):
    """
    Encontra e agrupa arquivos de cena EnMAP (VNIR, SWIR e máscaras) recursivamente.

    Args:
        base_path (str): O caminho do diretório base para procurar os arquivos EnMAP.

    Returns:
        list: Uma lista de dicionários, onde cada dicionário representa uma cena.
    """
    from collections import defaultdict
    scenes = defaultdict(dict)
    search_pattern = os.path.join(base_path, '**', '*.TIF')
    all_files = glob.glob(search_pattern, recursive=True)

    for f in all_files:
        filename = os.path.basename(f)
        scene_base_name = '_'.join(filename.split('_')[:4])

        if 'SPECTRAL_IMAGE_VNIR' in filename:
            scenes[scene_base_name]['vnir_img'] = f
        elif 'SPECTRAL_IMAGE_SWIR' in filename:
            scenes[scene_base_name]['swir_img'] = f
        elif 'QL_PIXELMASK_VNIR' in filename:
            scenes[scene_base_name]['vnir_mask'] = f
        elif 'QL_PIXELMASK_SWIR' in filename:
            scenes[scene_base_name]['swir_mask'] = f

    valid_scenes = [scene for scene in scenes.values() if 'vnir_img' in scene and 'swir_img' in scene]
    print(f"Encontradas {len(valid_scenes)} cenas EnMAP L1B completas (VNIR+SWIR).")
    return valid_scenes

def load_enmap_scene_as_array(scene_files):
    """
    Carrega uma cena EnMAP completa (VNIR + SWIR) como um cubo de dados NumPy,
    aplicando as máscaras de pixel.

    Args:
        scene_files (dict): Dicionário com os caminhos para os arquivos da cena.

    Returns:
        tuple: (cubo_dados, metadados) ou (None, None) em caso de falha.
    """
    try:
        with rasterio.open(scene_files['vnir_img']) as vnir_src, \
             rasterio.open(scene_files['swir_img']) as swir_src:

            if vnir_src.crs != swir_src.crs or vnir_src.transform != swir_src.transform:
                print(f"  AVISO: CRS ou Transform não correspondem entre VNIR e SWIR. Pulando cena.")
                return None, None

            vnir_rad = vnir_src.read()
            swir_rad = swir_src.read()
            full_radiance = np.vstack((vnir_rad, swir_rad)).astype(np.float32)
            metadata = vnir_src.meta.copy()

        full_mask = None
        if 'vnir_mask' in scene_files and 'swir_mask' in scene_files:
            with rasterio.open(scene_files['vnir_mask']) as vnir_mask_src, \
                 rasterio.open(scene_files['swir_mask']) as swir_mask_src:
                vnir_mask = vnir_mask_src.read(1)
                swir_mask = swir_mask_src.read(1)
                # Um pixel é inválido se for inválido em VNIR OU SWIR (máscara != 0)
                combined_mask = np.logical_or(vnir_mask != 0, swir_mask != 0)
                full_mask = np.broadcast_to(combined_mask, full_radiance.shape)

        if full_mask is not None:
            # Aplica a máscara, substituindo pixels inválidos por NaN
            full_radiance[full_mask] = np.nan
            print("  Máscara de pixels VNIR+SWIR aplicada.")
        else:
            print("  Nenhuma máscara de pixel encontrada ou aplicada.")

        # Atualiza metadados para o cubo combinado
        metadata.update(count=full_radiance.shape[0], dtype=str(full_radiance.dtype))
        return full_radiance, metadata

    except Exception as e:
        print(f"  ERRO ao carregar a cena: {e}")
        return None, None

print("Funções auxiliares definidas.")

In [ ]:
# --- 1. Definir Caminhos e Carregar Dados ---
drive_base_path = '/content/drive/My Drive/'
enmap_folder_name = 'ENMAP'
enmap_base_path = os.path.join(drive_base_path, enmap_folder_name)

enmap_scenes = find_enmap_scenes(enmap_base_path)

if not enmap_scenes:
    print("Nenhuma cena EnMAP encontrada. Verifique o caminho e a estrutura de pastas.")
else:
    # Carrega a primeira cena encontrada
    scene_name = os.path.basename(enmap_scenes[0]['vnir_img']).split('_ENMAP')[0]
    print(f"\\nProcessando cena: {scene_name}")
    radiance_cube, meta = load_enmap_scene_as_array(enmap_scenes[0])

    if radiance_cube is not None:
        # Transpõe para o formato (linhas, colunas, bandas) que a biblioteca `spectral` usa
        radiance_cube = np.transpose(radiance_cube, (1, 2, 0))

        # Remove pixels com NaN (mascarados)
        nan_mask = np.isnan(radiance_cube).any(axis=2)
        valid_pixels = radiance_cube[~nan_mask]
        print(f"Dimensões do cubo de dados: {radiance_cube.shape}")
        print(f"Número de pixels válidos: {valid_pixels.shape[0]}")

        # --- 2. Pré-processamento ---
        print("\\nETAPA 2: PRÉ-PROCESSAMENTO")
        hsi_smoothed = savgol_filter(valid_pixels, 11, 3, axis=1)
        print("Filtro Savitzky-Golay aplicado aos espectros.")

        # --- 3. Extração de Endmembers ---
        print("\\nETAPA 3: EXTRAÇÃO DE ENDMEMBERS")
        # Estima o número de endmembers (espectros puros)
        num_endmembers = spectral.hysime(hsi_smoothed.T)[0]
        print(f"Número de endmembers estimado (HySime): {num_endmembers}")

        # Encontra os pixels mais puros (PPI)
        ppi_indices = spectral.ppi(hsi_smoothed, num_endmembers)
        print("Índices dos pixels mais puros encontrados via PPI.")

        # Extrai os endmembers usando os índices do PPI como entrada para o N-FINDR
        endmembers = spectral.nfindr(hsi_smoothed, ppi_indices)[0]
        print(f"Endmembers extraídos via N-FINDR. Dimensões: {endmembers.shape}")

        # --- 4. Desmistura Espectral ---
        print("\\nETAPA 4: DESMISTURA ESPECTRAL")
        # Estima as abundâncias usando FCLS
        abundances = spectral.fcls(hsi_smoothed, endmembers)
        print(f"Abundâncias estimadas via FCLS. Dimensões: {abundances.shape}")

        # --- 5. Visualização dos Resultados ---
        print("\\nETAPA 5: VISUALIZAÇÃO")

        # Plotar os endmembers extraídos
        plt.figure(figsize=(10, 6))
        for i in range(num_endmembers):
            plt.plot(endmembers[i], label=f'Endmember {i + 1}')
        plt.title('Espectros dos Endmembers Extraídos', fontsize=16)
        plt.xlabel('Número da Banda')
        plt.ylabel('Radiância (escalonada)')
        plt.legend()
        plt.grid(True)
        plt.show()

        # Criar mapas de abundância
        abundance_maps = np.zeros((radiance_cube.shape[0] * radiance_cube.shape[1], num_endmembers))
        abundance_maps[~nan_mask.flatten()] = abundances
        abundance_maps = abundance_maps.reshape(radiance_cube.shape[0], radiance_cube.shape[1], num_endmembers)

        # Plotar os mapas de abundância
        fig, axes = plt.subplots(1, num_endmembers, figsize=(5 * num_endmembers, 5))
        if num_endmembers == 1: axes = [axes] # Garante que `axes` seja iterável
        fig.suptitle('Mapas de Abundância Estimados', fontsize=16)
        for i, ax in enumerate(axes):
            im = ax.imshow(abundance_maps[:, :, i], cmap='viridis', vmin=0, vmax=1)
            ax.set_title(f'Endmember {i + 1}')
            ax.axis('off')
            fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

In [ ]:
import os
import glob
import rasterio
import rasterio.mask
import numpy as np
import folium
from folium.utilities import tempfile
import matplotlib.pyplot as plt # Only for temporary RGB creation if needed
import geopandas as gpd # Ensure geopandas is imported

# Assuming drive_base_path and enmap_folder_name are defined
drive_base_path = '/content/drive/My Drive/'
enmap_folder_name = 'ENMAP'
enmap_base_path = os.path.join(drive_base_path, enmap_folder_name)

# Assuming local_gdf (APA polygon) is available from previous steps
if 'local_gdf' in locals() and not local_gdf.empty:
    print("\nProceeding to visualize EnMAP images with APA polygon overlay...")

    # Define patterns for Level 1B spectral images (VNIR and SWIR)
    spectral_image_patterns = ['*SPECTRAL_IMAGE_VNIR.TIF', '*SPECTRAL_IMAGE_SWIR.TIF']

    enmap_l1b_spectral_files = []

    print(f"Searching for EnMAP Level 1B spectral image files in subdirectories within: {enmap_base_path}")

    try:
        # Walk through the base directory and its subdirectories
        for root, dirs, files in os.walk(enmap_base_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Check for spectral image files
                for pattern in spectral_image_patterns:
                    if file.endswith(pattern.split('*')[-1]) and pattern.replace('*', '') in file:
                        enmap_l1b_spectral_files.append(file_path)
                        break # Found a spectral file


        print(f"\nFound {len(enmap_l1b_spectral_files)} EnMAP Level 1B spectral image files.")

        if not enmap_l1b_spectral_files:
            print("No EnMAP Level 1B spectral image files found matching patterns. Skipping visualization.")
        else:
            # Create a Folium map centered on the APA polygon
            # Get the bounds of the polygon to center the map
            bounds = local_gdf.total_bounds
            center_lat = (bounds[1] + bounds[3]) / 2
            center_lon = (bounds[0] + bounds[2]) / 2

            m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

            # Add the APA polygon to the map
            folium.GeoJson(local_gdf, name="APA Chapada do Araripe").add_to(m)

            # Add a Layer Control to toggle layers
            folium.LayerControl().add_to(m)


            # Process and add each EnMAP image as a raster layer
            processed_image_count = 0
            for enmap_file_path in enmap_l1b_spectral_files:
                 print(f"\nProcessing EnMAP L1B file for visualization: {os.path.basename(enmap_file_path)}")

                 try:
                     with rasterio.open(enmap_file_path) as src:
                         # Read the raster data and get georeferencing info
                         raster_data = src.read()
                         raster_crs = src.crs
                         raster_transform = src.transform
                         img_height, img_width = raster_data.shape[1:]
                         num_bands = raster_data.shape[0]

                         print(f"  Raster loaded. Bands: {num_bands}, Shape: {img_height}x{img_width}")
                         print(f"  CRS: {raster_crs}")

                         # --- Create RGB Composite for Visualization ---
                         # EnMAP has many bands. Select illustrative bands for RGB composite.
                         # Assuming bands around 650nm (Red), 550nm (Green), 450nm (Blue).
                         # These are illustrative indices, adjust based on actual band wavelengths if known.
                         # Common indices for L1B VNIR/SWIR might be different from L2A.
                         # Let's try generic indices that are likely within the bands.

                         # Check if enough bands
                         if num_bands >= 3:
                              # Using illustrative indices for a false color composite
                              # Example: Red=NIR (~800nm), Green=Red (~650nm), Blue=Green (~550nm)
                              # Need to adjust indices based on EnMAP band numbers/wavelengths
                              # Let's use simple indices for now, assuming band 1 is first band
                              # Try bands that might correspond to R,G,B or a common false color:
                              # Assuming bands roughly correspond to:
                              # VNIR: Bands 1-91
                              # SWIR: Bands 92-224
                              # Let's pick bands that are likely within VNIR or SWIR ranges
                              # Example: Red=Band 60 (VNIR), Green=Band 30 (VNIR), Blue=Band 10 (VNIR) - adjust as needed!
                              # Or a false color: Red=Band 150 (SWIR), Green=Band 80 (VNIR), Blue=Band 30 (VNIR)

                              # Check if file is VNIR or SWIR to pick appropriate bands
                              if 'VNIR' in os.path.basename(enmap_file_path) and num_bands >= 60:
                                  red_band_idx = 60 - 1 # Illustrative VNIR band
                                  green_band_idx = 30 - 1 # Illustrative VNIR band
                                  blue_band_idx = 10 - 1 # Illustrative VNIR band
                                  # Ensure indices are within bounds
                                  red_band_idx, green_band_idx, blue_band_idx = np.clip([red_band_idx, green_band_idx, blue_band_idx], 0, num_bands - 1)
                                  print(f"  Using illustrative VNIR bands for RGB: {red_band_idx+1}, {green_band_idx+1}, {blue_band_idx+1}")
                                  rgb_bands = [red_band_idx, green_band_idx, blue_band_idx]

                              elif 'SWIR' in os.path.basename(enmap_file_path) and num_bands >= (150-92+1): # Assuming SWIR starts around band 92
                                   # Illustrative SWIR false color
                                   red_band_idx = 150 - 1 # Illustrative SWIR band
                                   green_band_idx = 80 - 1 # Illustrative VNIR band (might not be in this file if it's pure SWIR)
                                   blue_band_idx = 30 - 1 # Illustrative VNIR band (might not be in this file if it's pure SWIR)

                                   # Need to handle cases where VNIR and SWIR are in separate files.
                                   # If this is a pure SWIR file, can't use VNIR bands.
                                   # Let's use simple indices for SWIR if it's a SWIR file
                                   if num_bands >= 3:
                                        red_band_idx = num_bands - 1 # Last band
                                        green_band_idx = num_bands // 2 # Middle band
                                        blue_band_idx = 0 # First band
                                        print(f"  Using simple SWIR bands for RGB: {red_band_idx+1}, {green_band_idx+1}, {blue_band_idx+1}")
                                        rgb_bands = [red_band_idx, green_band_idx, blue_band_idx]
                                   else:
                                        print(f"  WARNING: Not enough bands ({num_bands}) in SWIR file for RGB composite.")
                                        continue # Skip this file


                              else: # Default or not clearly VNIR/SWIR
                                   if num_bands >= 3:
                                        red_band_idx = 2 # First 3 bands
                                        green_band_idx = 1
                                        blue_band_idx = 0
                                        print("  Using first 3 bands for RGB.")
                                        rgb_bands = [red_band_idx, green_band_idx, blue_band_idx]
                                   else:
                                        print(f"  WARNING: Not enough bands ({num_bands}) for RGB composite.")
                                        continue # Skip this file


                              # Read the selected bands
                              rgb_image_data = src.read(indexes=[b + 1 for b in rgb_bands]) # rasterio uses 1-based indexing

                              # Transpose to (height, width, bands) for normalization/plotting
                              rgb_image_data = np.transpose(rgb_image_data, (1, 2, 0))

                              # Normalize/clip values to 0-255 or 0-1 for visualization
                              # L1B radiance values can vary widely. Simple min/max scaling might work.
                              # Find min/max across all bands used for RGB
                              min_val = np.min(rgb_image_data)
                              max_val = np.max(rgb_image_data)
                              if max_val > min_val:
                                   rgb_image_normalized = (rgb_image_data - min_val) / (max_val - min_val)
                                   # Clip to 0-1 range
                                   rgb_image_normalized = np.clip(rgb_image_normalized, 0, 1)
                                   print(f"  Normalized RGB values from [{min_val:.2f}, {max_val:.2f}] to [0, 1].")
                              else:
                                   print("  WARNING: Min and max values are the same. Skipping normalization.")
                                   rgb_image_normalized = rgb_image_data # Use raw data

                              # Convert to uint8 (0-255) for Pillow compatibility if needed by add_child method, or keep as float 0-1
                              # Folium ImageOverlay can often take float 0-1
                              rgb_image_display = (rgb_image_normalized * 255).astype(np.uint8)


                              # --- Add Raster Overlay to Folium Map ---
                              # Need to get the bounds of the image in WGS84 (Lat/Lon)
                              # Reproject the corners of the image from raster CRS to WGS84
                              try:
                                   # Get corners in raster CRS
                                   corners_raster_crs = [
                                       rasterio.transform.xy(raster_transform, 0, 0), # Top-left
                                       rasterio.transform.xy(raster_transform, 0, img_width), # Top-right
                                       rasterio.transform.xy(raster_transform, img_height, 0), # Bottom-left
                                       rasterio.transform.xy(raster_transform, img_height, img_width) # Bottom-right
                                   ]

                                   # Create a GeoDataFrame from corners in raster CRS
                                   corners_gdf_raster_crs = gpd.GeoDataFrame(
                                       geometry=gpd.points_from_xy([c[0] for c in corners_raster_crs], [c[1] for c in corners_raster_crs]),
                                       crs=raster_crs
                                   )

                                   # Reproject corners to WGS84 (EPSG:4326)
                                   corners_gdf_wgs84 = corners_gdf_raster_crs.to_crs(epsg=4326)

                                   # Get the bounds in WGS84
                                   wgs84_bounds = corners_gdf_wgs84.total_bounds
                                   # Folium bounds are [[south, west], [north, east]] i.e., [[ymin, xmin], [ymax, xmax]]
                                   folium_bounds = [[wgs84_bounds[1], wgs84_bounds[0]], [wgs84_bounds[3], wgs84_bounds[2]]]
                                   print(f"  Image bounds in WGS84 for Folium: {folium_bounds}")


                                   # Add the image overlay to the map
                                   # Use the normalized float image (0-1) or uint8 (0-255) - uint8 is safer
                                   folium.raster_layers.ImageOverlay(
                                       image=rgb_image_display, # Use uint8 image
                                       bounds=folium_bounds,
                                       origin='upper', # 'upper' for raster data
                                       name=os.path.basename(enmap_file_path).replace('.TIF', '') # Layer name
                                   ).add_to(m)
                                   print("  Image overlay added to map.")
                                   processed_image_count += 1


                              except Exception as geo_e:
                                   print(f"  ERROR reprojecting or getting bounds for Folium overlay: {geo_e}. Skipping visualization for this file.")
                                   continue # Skip to next file if georeferencing fails


                         else:
                              print(f"  WARNING: Not enough bands ({num_bands}) for RGB composite. Skipping visualization for this file.")
                              continue # Skip this file

                 except Exception as e:
                     print(f"  ERRO loading or processing EnMAP L1B file for visualization ({os.path.basename(enmap_file_path)}): {e}")
                     # Continue to the next file even if one fails


            # Display the map if at least one image was processed
            if processed_image_count > 0:
                print("\nDisplaying map with EnMAP images and APA polygon overlay.")
                display(m)
            else:
                print("\nNo EnMAP images were successfully processed for visualization.")


    except FileNotFoundError:
        print(f"Error: The base folder '{enmap_base_path}' was not found.")
        print("Please ensure the 'ENMAP' folder exists directly within 'My Drive' or update the 'enmap_folder_name' variable.")
    except Exception as e:
        print(f"An error occurred during EnMAP L1B file search or processing for visualization: {e}")


else:
    print("\nDataFrame 'local_gdf' (APA polygon) not found or empty. Skipping visualization.")